# Documentazione API OpenAI
---

### In questa documentazione vedremo come utilizzare le API di OpenAI nel linguaggio python con l'utilizzo di Redis come database a vettori.

Per prima dobbiamo sapere cos'è un database vettoriale e qual è il suo scopo: 
I database vettoriali sono database specializzati che hanno lo scopo di archiviare e gestire enormi quantità di dati ad alta dimensione sotto forma di vettori.\
I vettori sono rappresentazioni di dati matematici che descrivono oggetti in base alle loro diverse caratteristiche o qualità.
In questa documetnazione come database vettoriale andremo ad utilizzare Redis (Remote Dictionary Server) che permette attraverso il salvataggio dei dati nella memoria cache delle prestazioni migliori [per approfondire Redis](https://redis.io/).

Redis girerà in un container (un container è un’unità di software che racchiude l’applicazione e tutte le sue dipendenze, isolandola dal sistema operativo e dall’hardware sottostante).\
Per creare il container utilizzeremo un altrò software chiamato Docker.

Per prima cosa andiamo ad impostare il nostro Redis:
- Creiamo un cartella chiamata *redis.conf*.
- Creiamo un file *docker-compose.yml*.



All'interno del file *docker-compose.yml* inseriamo:


```yaml
version: '3.7'
services:

  vector-db:
    image: redis/redis-stack:latest
    ports:
      - 6379:6379
      - 8001:8001
    environment:
      - REDISEARCH_ARGS=CONCURRENT_WRITE_MODE
    volumes:
      - vector-db:/var/lib/redis
      - ./redis.conf:/usr/local/etc/redis/redis.conf
    healthcheck:
      test: ["CMD", "redis-cli", "-h", "localhost", "-p", "6379", "ping"]
      interval: 2s
      timeout: 1m30s
      retries: 5
      start_period: 5s

volumes:
  vector-db:
```

Fatto questo, quando andremo ad avviare il nostro programma basterà fare `docker compose up` per far partire Redis.

---

Per utilizzare le API di OpenAI bisogna prendere la KEY che è creabile a questo [link](https://platform.openai.com/account/api-keys).\
Adesso andiamo a creare un file *.env* e al suo interno scriviamo creiamo una variabile d'ambiente:\
`OPENAI_API_KEY = "OPENAI_API_KEY"`

Per prima cosa andiamo ad installare tutte le librerie che utilizzeremo:
- `pip install openai`
- `pip install redis`
- `pip install wget`
- `pip install pandas`
- `pip install fire`
- `pip install pypdf`
- `pip install python-dotenv`

O anche semplicemente:

- `pip install openai redis wget pandas fire pypdf python-dotenv`

Andiamo quindi ad importare le nostre librerie:

In [ ]:
import numpy as np
import openai
from pypdf import PdfReader
from redis.commands.search.field import TextField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
import redis
from dotenv import load_dotenv

Importiamo la nostra variabile d'ambiente contenuta nel file *.env*:

In [ ]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

Andiamo a definire i parametri per connetterci a Redis:

In [ ]:
INDEX_NAME = "embeddings-index"           # name of the search index
PREFIX = "doc"                            # prefix for the document keys
# distance metric for the vectors (ex. COSINE, IP, L2)
DISTANCE_METRIC = "COSINE"

REDIS_HOST = "localhost"
REDIS_PORT = 6379
REDIS_PASSWORD = "" 

redis_client = redis.Redis(
            host=REDIS_HOST,
            port=REDIS_PORT,
            password=REDIS_PASSWORD
        )


#### Adesso creiamo le funzioni che utilizzeremo nel nostro programma:

Questa funzione prende il *file.pdf* che gli viene passato in input, e restituisce una lista di dizionari aventi:
- Id
---
- Embredding: è la rappresentazione numerica di un testo e serve per misurare la correlazione delle stringhe di testo
---
- Text: piccole porzioni di testo prese dal file in input

In [ ]:
def pdf_to_embeddings(pdf_path: str, chunk_length: int = 550):
        # Read data from pdf file and split it into chunks
        reader = PdfReader(pdf_path)
        chunks = []
        for page in reader.pages:
            text_page = page.extract_text()
            chunks.extend([text_page[i:i+chunk_length].replace('\n', '')
                          for i in range(0, len(text_page), chunk_length)])

        # Create embeddings
        response = openai.Embedding.create(
            model='text-embedding-ada-002', input=chunks)
        return [{'id': value['index'], 'vector':value['embedding'], 'text':chunks[value['index']]} for value in response['data']]

Questa funzione ritorna una lista di dizionari come per la funzione che rigurda i *pdf* ma il testo in input è un *file.txt*:

In [ ]:
def txt_to_embeddings(text, chunk_length: int = 250):
        # Read data from pdf file and split it into chunks
        text = open(text, "r")
        text = text.read()

        chunks = []
        
        chunks.extend([text[i:i+chunk_length].replace('\n', '')
                       for i in range(0, len(text), chunk_length)])

        # Create embeddings
        response = openai.Embedding.create(
            model='text-embedding-ada-002', input=chunks)
        return [{'id': value['index'], 'vector':value['embedding'], 'text':chunks[value['index']]} for value in response['data']]

Facciamo lo stesso passando in input un *url*, per farlo dobbiamo però installare ed importare nuove librerie:
- `pip install beautifulsoup4`

In [ ]:
import requests 
from bs4 import BeautifulSoup
import re

def url_to_embeddings(url, chunk_length: int = 250):
        # Read data from pdf file and split it into chunks

        def format_url(testo):
            pattern = r"<.*?>"  # Pattern per cercare "<" seguito da qualsiasi carattere, incluso il newline, fino a ">"
            testo_senza_angolari = re.sub(pattern, "", testo)  # Rimuovi i match del pattern dal testo
            return testo_senza_angolari




        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        body = soup.find('body')
        content = str(soup.find_all("p"))
    
        text = format_url(content)


        chunks = []
        
        chunks.extend([text[i:i+chunk_length].replace('\n', '')
                       for i in range(0, len(text), chunk_length)])

        # Create embeddings
        response = openai.Embedding.create(
            model='text-embedding-ada-002', input=chunks)
        return [{'id': value['index'], 'vector':value['embedding'], 'text':chunks[value['index']]} for value in response['data']]

Prima di caricare il testo su Redis assicuriamoci che siano già stati forniti altri testi, nel caso ci fossero altri dati andiamo a cancellarli:

In [ ]:
def drop_redis_data(index_name: str = INDEX_NAME):
        try:
            redis_client.flushdb()
            print('Index dropped')
        except:
            # Index doees not exist
            print('Index does not exist')

Fatto questo carichiamo la lista di dizionari su Redis

In [ ]:
def load_data_to_redis(embeddings):
        # Constants
        vector_dim = len(embeddings[0]['vector'])  # length of the vectors
        
		# Initial number of vectors
        vector_number = len(embeddings)

        # Define RediSearch fields
        text = TextField(name="text")
        text_embedding = VectorField("vector",
                                     "FLAT", {
                                         "TYPE": "FLOAT32",
                                         "DIM": vector_dim,
                                         "DISTANCE_METRIC": "COSINE",
                                         "INITIAL_CAP": vector_number,
                                     }
                                     )
        fields = [text, text_embedding]

        # Check if index exists
        try:
            redis_client.ft(INDEX_NAME).info()
            print("Index already exists")
        except:
            # Create RediSearch Index
            redis_client.ft(INDEX_NAME).create_index(
                fields=fields,
                definition=IndexDefinition(
                    prefix=[PREFIX], index_type=IndexType.HASH)
            )

        for embedding in embeddings:
            key = f"{PREFIX}:{str(embedding['id'])}"
            embedding["vector"] = np.array(
                embedding["vector"], dtype=np.float32).tobytes()
            redis_client.hset(key, mapping=embedding)
        print(
            f"Loaded {redis_client.info()['db0']['keys']} documents in Redis search index with name: {INDEX_NAME}")

Una volta salvati i dati dobbiamo estrarre le parole chiavi dalla domanda dell'utente, per poi successivamente fare l'embedding di quest'ultime.\
Per farlo utilizziamo il modello *gpt-3.5-turbo* di OpenAI:

In [ ]:
def get_intent(user_question: str):
         # call the openai ChatCompletion endpoint
         response = openai.ChatCompletion.create(
         model="gpt-3.5-turbo",
         messages=[
               {"role": "user", "content": f'Extract the keywords from the following question: {user_question}'+
                 'Do not answer anything else, only the keywords.'}
            ]
         )
         

         # extract the response
         return (response['choices'][0]['message']['content'])

Prese le parole chiave dela domanda, facciamo l'embedding di quest'ultime e le confrontiamo con i nostri dati presenti in Redis.\
Facendo questo saremo in grado di prendere la porzione di testo nella quale è presente la risposta alla domanda:

In [ ]:
def search_redis(user_query: str,
                     index_name: str = "embeddings-index",
                     vector_field: str = "vector",
                     return_fields: list = ["text", "vector_score"],
                     hybrid_fields="*",
                     k: int = 20,
                     print_results: bool = False,
                     ):
        # Creates embedding vector from user query
        embedded_query = openai.Embedding.create(input=user_query,
                                                 model="text-embedding-ada-002",
                                                 )["data"][0]['embedding']
        # Prepare the Query
        base_query = f'{hybrid_fields}=>[KNN {k} @{vector_field} $vector AS vector_score]'
        query = (
            Query(base_query)
            .return_fields(*return_fields)
            .sort_by("vector_score")
            .paging(0, k)
            .dialect(2)
        )
        params_dict = {"vector": np.array(
            embedded_query).astype(dtype=np.float32).tobytes()}
        # perform vector search
        results = self.redis_client.ft(index_name).search(query, params_dict)
        if print_results:
            for i, doc in enumerate(results.docs):
                score = 1 - float(doc.vector_score)
                print(f"{i}. {doc.text} (Score: {round(score ,3) })")
        return [doc['text'] for doc in results.docs]

Per avere la risposta alla nostra domanda passiamo al modello *gpt-3.5-turbo* la nostra domanda e il testo dalla quale prendere le informazioni per rispondere:

In [ ]:
def generate_response(facts, user_question):
         # call the openai ChatCompletion endpoint
         response = openai.ChatCompletion.create(
         model="gpt-3.5-turbo",
         messages=[
               {"role": "user", "content": 'Based on the FACTS, give an answer to the QUESTION.'+ 
                f'QUESTION: {user_question}. FACTS: {facts}'}
            ]
         )

         

         # extract the response
         return (response['choices'][0]['message']['content'])

#### Testiamo il nostro programma

PDF:

In [ ]:
# Example pdf
pdf = 'giochi.pdf'


# Drop all data from redis if needed
data_service.drop_redis_data()




# Load data from pdf to redis
data = data_service.pdf_to_embeddings(pdf)

data_service.load_data_to_redis(data)